In [1]:
import os

import torch
import torch.nn as nn
from torch.utils.data import DataLoader, random_split
from torchvision.datasets import Food101
from torchvision.transforms import transforms
from modelpipe import HybridVitNet

from tqdm import tqdm
import matplotlib.pyplot as plt

In [2]:
os.getcwd()

'c:\\Users\\amany\\OneDrive\\Desktop\\projects\\genAi\\VitModel\\Main\\Notebooks'

In [3]:
PATH = 'c:\\Users\\amany\\OneDrive\\Desktop\\projects\\genAi\\VitModel\\Main'
ds_path = os.path.join(PATH, 'datasets')

image_size = (512, 512)
batch_size = 32
EPOCHS = 15
CLASSES = 101

g = torch.Generator(device="cpu")
device = torch.device('cuda' if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [4]:
normalizer = transforms.Normalize(mean=(0.485, 0.456, 0.406),std=(0.229, 0.224, 0.225),)
train_transform = transforms.Compose([transforms.Resize(image_size), transforms.ToTensor(), normalizer])
test_transform = transforms.Compose([transforms.ToTensor(),  normalizer])

In [5]:
trainset = Food101(ds_path, split='train', transform=train_transform)
testset = Food101(ds_path, split='test', transform=test_transform)

In [6]:
def data_loader(dataset, batch_size=batch_size, shuffle=True, cpu=2, prefetch=4, pm = True):
    return DataLoader(dataset, batch_size=batch_size, shuffle=shuffle,\
                       num_workers=cpu, prefetch_factor=prefetch, pin_memory=pm )

In [11]:
train_loader = data_loader(trainset)
test_loader = data_loader(testset, shuffle=False)

In [7]:
s_trainset_len = int(0.06* len(trainset))
remain = len(trainset) - s_trainset_len
s_trainset, _ = random_split(trainset, lengths=[s_trainset_len, remain], generator=g)
print("sample dataset:", len(s_trainset))

sample dataset: 4545


In [8]:
s_trainloader = data_loader(s_trainset)

In [9]:
model = HybridVitNet(num_classes=101).to(device)

In [10]:
model.initialize_weights()

In [ ]:
criterion = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

In [21]:
def model_trainer(model, data, epoch, loss_fn, optimizer_fn, validator=None):
    Loss = []

    model.train()

    for ep in range(epoch):
        batch_loss = 0
        sample_count = 0
        for x, y in tqdm(data, desc=f"Epoch {ep+1}"):
            x = x.to(device)
            y = y.to(device)
            optimizer_fn.zero_grad()
            
            out = model(x)
            loss = loss_fn(out, y)
    
    
            loss.backward()
            
            torch.nn.utils.clip_grad_norm_(model.parameters(), 0.5)
            optimizer_fn.step()
    
            sample_count+=1
            batch_loss += loss.detach().item()
            
        epoch_loss = batch_loss/sample_count
        Loss.append(epoch_loss)
        print(f"Loss: {epoch_loss}")

    return Loss
        

In [28]:
history = model_trainer(model, s_trainloader, EPOCHS, criterion, optimizer)

Epoch 1:   0%|          | 0/143 [00:00<?, ?it/s]

Epoch 1:  10%|▉         | 14/143 [01:36<14:53,  6.93s/it] 


KeyboardInterrupt: 